<a href="https://colab.research.google.com/github/aaronjyang/transformers-testing/blob/main/quality_degradation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load model and necessary imports

In [2]:
# Pip installs
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.0 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-mmlmnnid
  Running command git clone --filte

In [3]:
# Imports
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
import torch
from PIL import Image
import numpy as np
import av
import numpy as np
from huggingface_hub import hf_hub_download
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [4]:
# Load Model

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")
model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto'
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

# Load videos


In [17]:
def read_video_pyav(path, indices):
    '''
    Decode the video with PyAV decoder.

    Args:
        path (str): Path to the video file.
        indices (List[int]): List of frame indices to decode.

    Returns:
        np.ndarray: np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    container = av.open(path)
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [18]:
qualities = ["1080", "720", "480", "360", "240", "144"]
videos = {}
indices = np.arange(0, 51)
for quality in qualities:
  video = read_video_pyav("/content/quality_degradation/" + quality + "p10fps.mp4", indices)
  videos[quality] = video


In [35]:
randVideo = read_video_pyav("/content/000166.mp4", indices)

# Test videos (optional)


In [21]:
# np array with shape (frames, height, width, channels)
video = randVideo

fig = plt.figure()
im = plt.imshow(video[0,:,:,:])

plt.close() # this is required to not display the generated image

def init():
    im.set_data(video[0,:,:,:])

def animate(i):
    im.set_data(video[i,:,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
                               interval=100)
HTML(anim.to_html5_video())

# Misc. Functions

In [8]:
def generate_response(conversation, video, max_new_tokens = 100, do_sample= True, top_p = 0.9):
  prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
  inputs = processor([prompt], videos=video, padding=True, return_tensors="pt").to(model.device)
  generate_kwargs = {"max_new_tokens": max_new_tokens, "do_sample": do_sample, "top_p": top_p}
  output = model.generate(**inputs, **generate_kwargs)
  generated_text = processor.batch_decode(output, skip_special_tokens=True)
  return generated_text[0][len(prompt) - 7 + 1:]

# Experiment

In [12]:
# Create the template for the conversation
conversation = [
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "Does a car accident occur in this video"},
              {"type": "video"},
              ],
      },
]

In [13]:
for (quality, video) in videos.items():
  print((np.shape(video)))
  print(generate_response(conversation, video))

(50, 1080, 1920, 3)
Yes, a car accident can be seen occurring in this video. A red car is driving towards the camera when suddenly, it crashes with a silver car, causing both vehicles to spin off the road and into a ditch. The scene looks quite chaotic, with people's heads resting on the steering wheel and the red car appearing to have been flipped over. The video shows that the accident was severe and caused a major disruption on the roadway.
(50, 720, 1280, 3)
In this video, a car accident is taking place with two cars, possibly involving a red truck as well, with visible damage and debris on the road. Additionally, there is a person who is getting out of a car, and a pedestrian is crossing the street in the middle of the intersection.
(50, 480, 854, 3)
Yes, an accident appears to occur in the video. Two vehicles collide and one car gets towed away by a tow truck. Additionally, there are other cars present in the scene, including a white car with an item sitting in the back seat, a r

In [39]:
# Generate model-readable prompt
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
inputs = processor([prompt], videos=[randVideo], padding=True, return_tensors="pt").to(model.device)

In [40]:
generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.9}
output = model.generate(**inputs, **generate_kwargs)
generated_text = processor.batch_decode(output, skip_special_tokens=True)

In [41]:
print(generated_text)

['USER: \nDoes a car accident occur in this video ASSISTANT: The video captures an accident involving a car and a tow truck. In the video, we can see that a tow truck has rear-ended the car, which has caused a tire to fly out of the car and land on the road. There is a car following the tow truck, and we can see the tow truck itself is likely not moving or is moving at a very slow pace due to the accident. The video does not provide clear evidence of']
